# "Nationality" Data Cleaning
Cross Checking the validity of the gender and nationality API Mary is using with Ancestry datasets Tim has scraped (`Ancestry_1910`) for `1898_TrowBus dataset`

### Requirements

In [6]:
import pandas as pd

### Datasets

In [39]:
df1 = pd.read_csv('1898TrowBus.csv')[['Name', 'Addy_Clean', 'Nationality Final', 'nationality_mary']]
df1.columns = ['name', 'address', 'nationality_final', 'nationality_mary']
df1.head(1)

,name,address,nationality_final,nationality_mary
0,Abbott Charles W,548 W 39th,English,English


In [40]:
df2 = pd.read_csv('1913TrowBus.csv')[['Name', 'Addy_Clean', 'NATIONALITY final']]
df2.columns = ['name', 'address', 'nationality_final']
df2.head(1)

,name,address,nationality_final
0,Albanian Roumanian Co,261 W 37th,Albanian


In [104]:
df3 = pd.read_csv('Ancesrty_1910_Manhattan_Ward7.csv')[
    ['Name', 'First Name', 'Surname', 
     'Address', 'Street Name New', 'House Number New', 
     'Birthplace', 'Father\'s Birthplace', 'Mother\'s Birthplace']]
#[['Name', 'Address', 'Birthplace', 'Father\'s Birthplace', 'Mother\'s Birthplace']]

df3.columns = ['name', 'first', 'last', 
               'address', 'street', 'no',
               'birthplace', 'bpf', 'bpm']
#df3.columns = ['name', 'address', 'birthplace', 'birthplace_f', 'birthplace_m']

df3.head(1)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3050: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,name,first,last,address,street,no,birthplace,bpf,bpm
0,I Virginia Isola,I,Isola,Catherine Street 25,Catherine Street,25,Italy,Italy,Italy


In [100]:
df3.columns

Index(['DataID', 'RecordId', 'Name', 'First Name', 'Middle Name', 'Surname',
       'HouseHoldId', 'Street Name New', 'House Number New', 'Address',
       'PlaceId', 'Lon', 'Lat', 'Dwelling Number', 'Own or Rent',
       'Relationship', 'Is Head', 'Gender', 'Race', 'Age', 'Marital Status',
       'Number of Years of Present Marriage', 'Number of Children Born',
       'Number of Children Living', 'Birthplace', 'Father's Birthplace',
       'Mother's Birthplace', 'US Born', 'Immigration Year',
       'Naturalization Status', 'Self Native Tongue', 'Occupation',
       'Occupation Category', 'Industry', 'Self Employed or Not',
       'Out of Work on April 15', 'Number of Weeks Out of Work', 'Can Read',
       'Can Write', 'Attended School', 'Blind', 'Deaf and Dumb',
       'Enumeration District Number', 'Ward of City', 'Street Name',
       'House Number'],
      dtype='object')

### dataset1: `1898TrowBus`

How is the discrepancy between the final result and Mary's API?

In [68]:
diff = (df1['nationality_final'] != df1['nationality_mary'])
diff.value_counts()

False    1853
True      141
dtype: int64

This means there is only discrepancy in 141 pieces of data.

In [67]:
df1[diff]

,name,address,nationality_final,nationality_mary
2,Abelmont Eugenia,486 6th av,NaN,Spainish
29,Alten Henry,227 W Broadway,German,English
134,Beshawte K,434 6th av,NaN,English
138,Bette Brothers,20 Spring,NaN,English
147,Bittner John,2273 8th av,NaN,German
...,...,...,...,...
1742,+Still's J H Son,195 3rd av,NaN,NaN
1747,Streen & Liberman,154 Centre,NaN,NaN
1835,Velmahos Brothers,107 Greenwich,NaN,NaN
1837,Vermatten Jules,100 Broad,NaN,NaN


Inspecing the data, I realize that the discrepancy exists mostly not because they had different results, but because nationality_final did not have a result. 

In [75]:
real_diff = df1[diff]['nationality_final'].notnull()
real_diff.value_counts()

False    138
True       3
Name: nationality_final, dtype: int64

This means that only 3 of the discrepancy are actual discrepancies. 

In [76]:
df1[diff][real_diff]

,name,address,nationality_final,nationality_mary
29,Alten Henry,227 W Broadway,German,English
177,Bormann George,342 9th av,Gernan,German
1300,Muth Theodore P,389 West st,-,NaN


I want to cross-check these 3 data with Tim's Ancestry dataset.

In [124]:
df3[df3['first']=='Alten']

,name,first,last,address,street,no,birthplace,bpf,bpm
18268,Alten Majerovitz,Alten,Majerovitz,Pike Street 11,Pike Street,11,Russia,Russia,Russia


No, his last name is "Henry" so he is unlikely Russian, despite his name being Alten.

(fyi, Alten as a family name would likely be German, but as first name...?)

In [125]:
df3[df3['last']=='Henry']

,name,first,last,address,street,no,birthplace,bpf,bpm
886,William Henry,William,Henry,Madison Street 97,Madison Street,97,New York,Ireland,Ireland
887,Sarah Henry,Sarah,Henry,Madison Street 97,Madison Street,97,New York,Ireland,Ireland
888,William Henry,William,Henry,Madison Street 97,Madison Street,97,New York,New York,New York
28636,Harry Henry,Harry,Henry,Pelham Street 9,Pelham Street,9,Russia,Russia,Russia
72274,John P Henry,John,Henry,Scammel Street 58,Scammel Street,58,Sweden,Sweden,Sweden
72275,Bertha Henry,Bertha,Henry,Scammel Street 58,Scammel Street,58,Sweden,Sweden,Sweden
72276,Alex Henry,Alex,Henry,Scammel Street 58,Scammel Street,58,New York,Sweden,Sweden
72277,Victoria Henry,Victoria,Henry,Scammel Street 58,Scammel Street,58,New York,Sweden,Sweden
74105,Jane Henry,Jane,Henry,Gouverneur Street 91,Gouverneur Street,91,Ireland,Ireland,Ireland


"Henry" is too common of a last name to get any accurate results. 

1. Wait... there is "Alten" as first name being Russian and "Henry" as last name being Russian... could "Alten Henry" be Russian???

2. "Henry" also seems to be a common Swedish last name?? But the descrepancy was between German and English...

3. I realized signs of ethnic clusters? The same birthplace lived on the same stree? But also they have the exact same address so they were probably just a family...?

In [133]:
df3[(df3['street']=='Broadway') & (df3['no']>130)]

,name,first,last,address,street,no,birthplace,bpf,bpm
37089,Isidore Levin,Isidore,Levin,Broadway 181,Broadway,181,United States,Germany,Germany
37090,Rosie Levin,Rosie,Levin,Broadway 181,Broadway,181,Germany,Germany,Germany
37091,Pearlie Levin,Pearlie,Levin,Broadway 181,Broadway,181,United States,United States,Germany
37092,Emanuel Levin,Emanuel,Levin,Broadway 181,Broadway,181,United States,United States,Germany


The Levin family was the only German family living on Broadway Street, there is no result between number 130 to 227 (which is what we are looking for). 

The ethnic cluster doesn't really make sense becasue they seem to be families instead of clusters, but I choose to be that "Alten Henry" is more likely German... because there has been few indicators of him being English so far?

In [135]:
df3[df3['first']=='Muth']

,name,first,last,address,street,no,birthplace,bpf,bpm


In [136]:
df3[df3['street']=='West']

,name,first,last,address,street,no,birthplace,bpf,bpm


No data about "Muth Theodore P".

### dataset2: `1913TrowBus`

In [139]:
df2[df2['nationality_final'].isnull()]

,name,address,nationality_final
3672,Aberdeen,17 W 32nd,NaN
3673,ABINGOON HOTEL,7 Abingdon sq,NaN
3674,Acropolis,519 W 123rd,NaN
3675,Adelphi,470 W 23rd,NaN
3676,Albemarle,1 W 24th,NaN
...,...,...,...
5414,Zimmer Jacob,815 Westchester av,NaN
5415,Zimmerman Isaac,159 Monroe,NaN
5416,Zimmermann Agnes,424 Amsterdam av,NaN
5417,Zneimer Herman,2229 2nd av,NaN
